In [3]:
import pandas as pd
import numpy as np

What does the alert querry itself returns??

# Alert Criteria for Suspect Merchants

The following criteria are used to identify suspect merchants for generating an alert:

## 1. Transações de Risco
- O comerciante deve ter transações de risco com tipos específicos: `'velocity_check'`, `'location_not_allowed'`, `'cp_location'`, ou `'cnp_location'`.

## 2. Transações Internacionais
- As transações de risco devem ser internacionais (`t7.international IS TRUE`).

## 3. Métodos de Captura de Transação
- As transações válidas devem ter métodos de captura `'mpos'`, `'contactless'`, ou `'emv'`.

## 4. Validação do Cartão
- As transações devem ter um `card_token_id` não nulo.

## 5. Exclusão de Especificações Bancárias
- O nome do banco emissor (`issuer`) deve ser diferente de `'caixa economica federal'`.

## 6. Estado do Comerciante
- O estado do comerciante deve ser `'approved'`.

## 7. Período de Transação
- As transações devem ter ocorrido nos últimos 3 dias (`DATETIME(t.created_at) >= CURRENT_DATE() - 1`).

## 8. Recência da Conta
- O comerciante não deve ser um novo cliente, ou seja, a conta deve ter sido criada há mais de 90 dias (`creation > 90`).

## 9. Concentração de BIN Internacional
- O comerciante não deve ter uma alta concentração de transações com emissores internacionais. A condição é que a proporção de transações internacionais não seja maior que 50% do total de transações.

## 10. Contagem de Transações em Janelas de Tempo
Dependendo do tempo de criação da conta (`creation`) e se o comerciante é pessoa física (`is_PF`), a contagem de transações em janelas de tempo de 3, 10, 20 e 40 minutos deve atender aos seguintes critérios:
- **Para Pessoa Física (is_PF = TRUE)**:
  - Contagem de transações (int_3min, int_10min, int_20min, int_40min) dentro dos limites definidos para criação maior ou menor que 90 dias.
- **Para Pessoa Jurídica (is_PF = FALSE)**:
  - Contagem de transações dentro dos limites definidos para criação maior ou menor que 180 dias.

## 11. Exclusões Específicas
- O comerciante não deve estar nas listas de exceções específicas (`monitoring_not_to_analyse` e `inkman_score`).

Esses critérios são aplicados para identificar comportamentos suspeitos de fraude ou abuso nas transações, e os comerciantes que atendem a esses critérios são marcados com um alerta detalhado e categorizado como suspeito.


Frist I'll take a look at the alerts that are from *PJ* and were created in the *last 180 days.*
Then I'll make the same analysis for *PF* that were create in the last *180 days*.
Try to find patterns where the alert works well, and them "fix" the querry.


In [4]:
data_cnpj = pd.read_csv("data/cp_international_bin_cnpj.csv")
data_cpf = pd.read_csv("data/cp_international_bin_cpf.csv")

In [5]:
data_cnpj.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 17 columns):
 #   Column                                      Non-Null Count  Dtype  
---  ------                                      --------------  -----  
 0   user_id                                     104 non-null    int64  
 1   status                                      104 non-null    object 
 2   status_reason                               25 non-null     object 
 3   status_1                                    104 non-null    object 
 4   active_days_since_aquisition_from_merchant  104 non-null    int64  
 5   approved_transactions_count_from_merchant   104 non-null    int64  
 6   approved_amount_sum_from_merchant           104 non-null    float64
 7   cbk_tpv_ratio                               8 non-null      float64
 8   total_billing_amount                        8 non-null      float64
 9   total_chargebacks                           8 non-null      float64
 10  paid_ratio    

In [6]:
total_cnpj = data_cnpj['status'].count()
active_cnpj = data_cnpj[data_cnpj['status'] == 'active'].count()[0]
blocked_cnpj = data_cnpj[data_cnpj['status'] == 'blocked'].count()[0]
total_cpf = data_cpf['status'].count()
active_cpf = data_cpf[data_cpf['status'] == 'active'].count()[0]
blocked_cpf = data_cpf[data_cpf['status'] == 'blocked'].count()[0]

/tmp/ipykernel_52593/2141584214.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  active_cnpj = data_cnpj[data_cnpj['status'] == 'active'].count()[0]
/tmp/ipykernel_52593/2141584214.py:3: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  blocked_cnpj = data_cnpj[data_cnpj['status'] == 'blocked'].count()[0]
/tmp/ipykernel_52593/2141584214.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  active_cpf = data_cpf[data_cpf['status'] == 'a

In [15]:
TP_cnpj_block = blocked_cnpj/total_cnpj*100
TP_cpf_block = blocked_cpf/total_cpf*100

In [17]:
print(f"In the last 90 days, out of {total_cnpj} CNPJ that were flagged by the alert. {blocked_cnpj} are still blocked and {active_cnpj} are active. {TP_cnpj_block:.2f}% are TP blocked.",'\n')

print(f"In the last 90 days, out of {total_cpf} CPF that were flagged by the alert. {blocked_cpf} are still blocked and {active_cpf} are active. {TP_cpf_block:.2f}% are TP blocked.",'\n')



In the last 90 days, out of 104 CNPJ that were flagged by the alert. 21 are still blocked and 82 are active. 20.19% are TP blocked. 

In the last 90 days, out of 322 CPF that were flagged by the alert. 153 are still blocked and 166 are active. 47.52% are TP blocked. 



It's very important to increase CNPJ precision, and to understand how to increase even further precision on CPF.